In [23]:
# Re-import necessary libraries and reload the data due to the reset of code execution state
import pandas as pd

# Define file paths
gps_data_path = r"E:\无锡公交\无锡锡山公交数据\21.6.15-6.21数据\T_Bus_Gps_2021.6.15.csv"
trans_data_path = r"E:\无锡公交\无锡锡山公交数据\21.6.15-6.21数据\T_Bus_Transaction_2021.6.15.csv"
stop_data_path = r"E:\无锡公交\无锡锡山公交数据\21.6.15-6.21数据\Phy_Stop.csv"

# Load the GPS data
gps_data = pd.read_csv(gps_data_path, parse_dates=['DateTime'])

# Load the transaction data
trans_data = pd.read_csv(trans_data_path, parse_dates=['DateTime'])

# Load the stop data with appropriate encoding for Chinese characters
stop_data = pd.read_csv(stop_data_path, encoding='GBK')

# Display the first few rows of each dataset to confirm loading
gps_data.head(), trans_data.head(), stop_data.head()

(            DateTime  Line_ID  Bus_ID         lng        lat  Speed  Azimuth
 0  2021-6-15 6:56:50        1   82676  120.544053  31.712185    0.0    229.0
 1  2021-6-15 6:56:50        1   82676  120.544053  31.712185    0.0    229.0
 2  2021-6-15 6:57:02        1   82676  120.544053  31.712185    0.0    229.0
 3  2021-6-15 6:57:02        1   82676  120.544053  31.712185    0.0    229.0
 4  2021-6-15 6:57:12        1   82676  120.544053  31.712185    0.0    229.0,
              DateTime      Card_ID  Price  Card_Type  Line_ID  Bus_ID
 0 2021-06-15 17:11:00  17285279105    1.2         80      626   65023
 1 2021-06-15 17:11:00   5211164905    1.2         80      626   65023
 2 2021-06-15 17:19:00  98834866221    1.2         80      626   65023
 3 2021-06-15 17:11:00  52091226632    1.2         80    73203   82388
 4 2021-06-15 17:11:00  68096054226    1.2         80    73203   82388,
    Phy_Stop_ID  Stop_Name   Lng_no_84  Lat_no_84 Line_Name  Line_ID_GPS  \
 0            0       梁溪大桥  

公交车是否到站识别，分批次执行

In [26]:
from scipy.spatial.distance import cdist
import numpy as np
def calculate_distances_in_batches(gps_coords, stop_coords, batch_size=500):
    num_batches = np.ceil(len(gps_coords) / batch_size).astype(int)
    min_distances = np.zeros(len(gps_coords))
    min_distance_idx = np.zeros(len(gps_coords), dtype=int)
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(gps_coords))
        
        # Calculate distance matrix for the current batch
        dist_matrix_batch = cdist(gps_coords[start_idx:end_idx], stop_coords)
        
        # Find the minimum distance and its index for each GPS record in the batch
        min_distances_batch = dist_matrix_batch.min(axis=1) * 1000  # Convert from kilometers to meters
        min_distance_idx_batch = dist_matrix_batch.argmin(axis=1)
        
        # Store the results
        min_distances[start_idx:end_idx] = min_distances_batch
        min_distance_idx[start_idx:end_idx] = min_distance_idx_batch
    
    return min_distances, min_distance_idx

# Calculate the minimum distances and their indices in batches
min_distances, min_distance_idx = calculate_distances_in_batches(gps_coords, stop_coords)

# Determine whether each GPS record is within 200 meters of its nearest stop
is_arrived = min_distances <= 200

# Add the nearest stop ID and arrival status to the GPS data
gps_data['Nearest_Stop_ID'] = stop_data.iloc[min_distance_idx]['Phy_Stop_ID'].values
gps_data['Is_Arrived'] = is_arrived

# Display some of the results to verify
gps_data[['DateTime', 'Line_ID', 'Bus_ID', 'lat', 'lng', 'Nearest_Stop_ID', 'Is_Arrived']].head(5)

,DateTime,Line_ID,Bus_ID,lat,lng,Nearest_Stop_ID,Is_Arrived
0,2021-6-15 6:56:50,1,82676,31.712185,120.544053,3158,True
1,2021-6-15 6:56:50,1,82676,31.712185,120.544053,3158,True
2,2021-6-15 6:57:02,1,82676,31.712185,120.544053,3158,True
3,2021-6-15 6:57:02,1,82676,31.712185,120.544053,3158,True
4,2021-6-15 6:57:12,1,82676,31.712185,120.544053,3158,True


公交车到站识别，与上一段相同功能，但运算效率比较低，适合demo演示

In [ ]:
# from scipy.spatial.distance import cdist
# import numpy as np

# # Define a function to convert lat/lon to Cartesian coordinates for distance calculation
# def latlon_to_cartesian(lat, lon):
#     # Convert latitude and longitude from degrees to radians
#     lat_rad = np.radians(lat)
#     lon_rad = np.radians(lon)
#     # Earth's radius in kilometers (approximate)
#     R = 6371
#     x = R * np.cos(lat_rad) * np.cos(lon_rad)
#     y = R * np.cos(lat_rad) * np.sin(lon_rad)
#     return x, y

# # Apply the conversion to both GPS and stop data
# gps_coords = np.vstack(latlon_to_cartesian(gps_data['lat'], gps_data['lng'])).T
# stop_coords = np.vstack(latlon_to_cartesian(stop_data['Lat'], stop_data['Lng'])).T

# # Calculate the distance matrix between GPS points and stops
# dist_matrix = cdist(gps_coords, stop_coords)

# # Find the minimum distance and its index for each GPS record
# min_distances = dist_matrix.min(axis=1) * 1000 # Convert from kilometers to meters
# min_distance_idx = dist_matrix.argmin(axis=1)

# # Determine whether each GPS record is within 200 meters of its nearest stop
# is_arrived = min_distances <= 200

# # Add the nearest stop ID and arrival status to the GPS data
# gps_data['Nearest_Stop_ID'] = stop_data.iloc[min_distance_idx]['Phy_Stop_ID'].values
# gps_data['Is_Arrived'] = is_arrived

# # Display some of the results to verify
# gps_data[['DateTime', 'Line_ID', 'Bus_ID', 'lat', 'lng', 'Nearest_Stop_ID', 'Is_Arrived']].head(5)

思路1，根据是否到站来匹配乘客上车站点

In [32]:
# Filter GPS data for records where the bus is considered to have arrived at a stop
arrived_gps_data = gps_data[gps_data['Is_Arrived'] == True]


# Function to find the nearest arrived GPS record for each transaction
def find_nearest_arrived_gps(row):
    # Filter GPS records by the same line and bus ID, and by time before the transaction
    relevant_gps_records = arrived_gps_data[(arrived_gps_data['Line_ID'] == row['Line_ID']) & 
                                            (arrived_gps_data['Bus_ID'] == row['Bus_ID']) & 
                                            (arrived_gps_data['DateTime'] <= row['DateTime'])]
    
    # If no relevant GPS record is found, return NaN
    if relevant_gps_records.empty:
        return pd.NA
    
    # Find the GPS record closest in time to the transaction
    closest_record = relevant_gps_records.iloc[(relevant_gps_records['DateTime'] - row['DateTime']).abs().argmin()]
    return closest_record['Nearest_Stop_ID']

gps_data['DateTime'] = pd.to_datetime(gps_data['DateTime'], errors='coerce', format='mixed')
trans_data['DateTime'] = pd.to_datetime(trans_data['DateTime'], errors='coerce', format='mixed')
# Apply the function to each transaction to assign the boarding stop ID
trans_data['Boarding_Stop_ID'] = trans_data.apply(find_nearest_arrived_gps, axis=1)

# Display the updated transaction data to verify boarding stop identification
trans_data[['DateTime', 'Card_ID', 'Line_ID', 'Bus_ID', 'Boarding_Stop_ID']].head()
trans_data

,DateTime,Card_ID,Price,Card_Type,Line_ID,Bus_ID,Boarding_Stop_ID
0,2021-06-15 17:11:00,17285279105,1.2,80,626,65023,713
1,2021-06-15 17:11:00,5211164905,1.2,80,626,65023,713
2,2021-06-15 17:19:00,98834866221,1.2,80,626,65023,713
3,2021-06-15 17:11:00,52091226632,1.2,80,73203,82388,1942
4,2021-06-15 17:11:00,68096054226,1.2,80,73203,82388,1942
...,...,...,...,...,...,...,...
49755,2021-06-15 19:13:00,1084027,0.0,1,713,85051,1906
49756,2021-06-15 19:15:00,70000094634,1.2,1,617,60616,1447
49757,2021-06-15 08:02:00,16627,1.2,1,703,82531,380
49758,2021-06-15 10:02:00,2116,1.2,1,609,65003,<NA>


思路2，根据数卡时间匹配最接近的公交车GPS记录，在根据找到的GPS记录中的最近站点作为乘客上车站点

In [38]:
# Function to find the nearest GPS record for each transaction
def find_nearest_gps_for_boarding(row):
    # Filter GPS records by the same line and bus ID
    relevant_gps_records = gps_data[(gps_data['Line_ID'] == row['Line_ID']) & 
                                    (gps_data['Bus_ID'] == row['Bus_ID'])]
    
    # If no relevant GPS record is found, return NaN
    if relevant_gps_records.empty:
        return pd.NA
    
    # Calculate time difference between transaction time and all relevant GPS records
    time_diffs = relevant_gps_records['DateTime'] - row['DateTime']
    
    # Find the index of the GPS record closest in time to the transaction (minimum absolute time difference)
    closest_idx = time_diffs.abs().idxmin()
    
    # Return the Nearest_Stop_ID of the closest GPS record
    return gps_data.loc[closest_idx, 'Nearest_Stop_ID']
gps_data['DateTime'] = pd.to_datetime(gps_data['DateTime'], errors='coerce', format='mixed')
trans_data['DateTime'] = pd.to_datetime(trans_data['DateTime'], errors='coerce', format='mixed')
# Apply the function to each transaction to assign the boarding stop ID based on nearest GPS record
trans_data['Boarding_Stop_ID'] = trans_data.apply(find_nearest_gps_for_boarding, axis=1)

# Display the updated transaction data to verify the new method of boarding stop identification
trans_data[['DateTime', 'Card_ID', 'Line_ID', 'Bus_ID', 'Boarding_Stop_ID']].head()
trans_data

,DateTime,Card_ID,Price,Card_Type,Line_ID,Bus_ID,Boarding_Stop_ID,DateTime_30Min
0,2021-06-15 17:11:00,17285279105,1.2,80,626,65023,709,2021-06-15 17:00:00
1,2021-06-15 17:11:00,5211164905,1.2,80,626,65023,709,2021-06-15 17:00:00
2,2021-06-15 17:19:00,98834866221,1.2,80,626,65023,706,2021-06-15 17:00:00
3,2021-06-15 17:11:00,52091226632,1.2,80,73203,82388,1942,2021-06-15 17:00:00
4,2021-06-15 17:11:00,68096054226,1.2,80,73203,82388,1942,2021-06-15 17:00:00
...,...,...,...,...,...,...,...,...
49755,2021-06-15 19:13:00,1084027,0.0,1,713,85051,1906,2021-06-15 19:00:00
49756,2021-06-15 19:15:00,70000094634,1.2,1,617,60616,221,2021-06-15 19:00:00
49757,2021-06-15 08:02:00,16627,1.2,1,703,82531,380,2021-06-15 08:00:00
49758,2021-06-15 10:02:00,2116,1.2,1,609,65003,709,2021-06-15 10:00:00


In [39]:
# Perform passenger boarding count statistics by grouping by the Boarding_Stop_ID
boarding_flow_statistics = trans_data.groupby('Boarding_Stop_ID').size().reset_index(name='Passenger_Count')

# Merge the boarding flow statistics with stop information to get more detailed information
boarding_flow_with_stop_info = pd.merge(boarding_flow_statistics, stop_data[['Phy_Stop_ID', 'Stop_Name']], 
                                        left_on='Boarding_Stop_ID', right_on='Phy_Stop_ID', how='left')

# Display the boarding flow statistics with stop names
boarding_flow_with_stop_info[['Stop_Name', 'Passenger_Count']].sort_values(by='Passenger_Count', ascending=False).head()

,Stop_Name,Passenger_Count
173,盛岸公交停车场,8568
314,金惠公交总站,1325
158,解放西路(县前街),881
321,奥林匹克花园,732
279,长安汽车站,693


In [40]:
# Adjust the DateTime in transaction data to 30-minute intervals for boarding statistics
trans_data['DateTime_30Min'] = trans_data['DateTime'].dt.floor('30T')

# Group by Boarding_Stop_ID and DateTime_30Min, then count passengers
boarding_flow_30min = trans_data.groupby(['Boarding_Stop_ID', 'DateTime_30Min']).size().reset_index(name='Passenger_Count_30Min')

# Merge the boarding flow statistics with stop information to get more detailed information
boarding_flow_30min_with_stop_info = pd.merge(boarding_flow_30min, stop_data[['Phy_Stop_ID', 'Stop_Name']], 
                                              left_on='Boarding_Stop_ID', right_on='Phy_Stop_ID', how='left')

# Display the boarding flow statistics with stop names for 30-minute intervals
boarding_flow_30min_with_stop_info[['Stop_Name', 'DateTime_30Min', 'Passenger_Count_30Min']].sort_values(by=['Stop_Name', 'DateTime_30Min']).head()

,Stop_Name,DateTime_30Min,Passenger_Count_30Min
7672,312国道(泰伯大道),2021-06-15 09:00:00,1
8046,312国道(泰伯大道),2021-06-15 10:30:00,1
7673,312国道(泰伯大道),2021-06-15 13:00:00,1
7674,312国道(泰伯大道),2021-06-15 16:00:00,2
8047,312国道(泰伯大道),2021-06-15 16:30:00,1


In [41]:
# Convert 'DateTime_30Min' to just the time for easier pivoting and filtering from 6 AM to 9 PM
boarding_flow_30min_with_stop_info['Time'] = boarding_flow_30min_with_stop_info['DateTime_30Min'].dt.time

# Filter the data to include only the time range from 6 AM to 9 PM
start_time = pd.to_datetime("06:00:00").time()
end_time = pd.to_datetime("21:00:00").time()
filtered_data = boarding_flow_30min_with_stop_info[(boarding_flow_30min_with_stop_info['Time'] >= start_time) & 
                                                    (boarding_flow_30min_with_stop_info['Time'] <= end_time)]

# Pivot the table to get the desired format
pivot_table = filtered_data.pivot_table(index='Stop_Name', columns='Time', values='Passenger_Count_30Min', fill_value=0).reset_index()


# Correcting the column flattening process without incorrect type check
pivot_table.columns = [col.strftime('%H:%M') if isinstance(col, pd.Timestamp) else col for col in pivot_table.columns]
# Convert passenger counts to integer
pivot_table = pivot_table.fillna(0)  # First fill NA values with 0 to avoid conversion errors
pivot_table.iloc[:, 1:] = pivot_table.iloc[:, 1:].astype(int)  # Convert all columns except the first (Stop_Name) to integer

# Display the restructured data with corrected column names
pivot_table.head()

,Stop_Name,06:00:00,06:30:00,07:00:00,07:30:00,08:00:00,08:30:00,09:00:00,09:30:00,10:00:00,...,16:30:00,17:00:00,17:30:00,18:00:00,18:30:00,19:00:00,19:30:00,20:00:00,20:30:00,21:00:00
0,312国道(泰伯大道),0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,904医院(兴源路),0.0,1.0,5.0,1.0,2.0,4.0,2.0,3.0,4.0,...,2.0,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
2,一汽锡柴,0.0,3.0,8.0,1.0,4.0,2.0,0.0,7.0,6.0,...,0.0,4.0,4.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0
3,丁村,0.0,0.0,8.0,5.0,1.0,3.0,6.0,2.0,0.0,...,4.0,6.0,9.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0
4,七坊桥,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
pivot_table

,Stop_Name,06:00:00,06:30:00,07:00:00,07:30:00,08:00:00,08:30:00,09:00:00,09:30:00,10:00:00,...,16:30:00,17:00:00,17:30:00,18:00:00,18:30:00,19:00:00,19:30:00,20:00:00,20:30:00,21:00:00
0,312国道(泰伯大道),0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,904医院(兴源路),0.0,1.0,5.0,1.0,2.0,4.0,2.0,3.0,4.0,...,2.0,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
2,一汽锡柴,0.0,3.0,8.0,1.0,4.0,2.0,0.0,7.0,6.0,...,0.0,4.0,4.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0
3,丁村,0.0,0.0,8.0,5.0,1.0,3.0,6.0,2.0,0.0,...,4.0,6.0,9.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0
4,七坊桥,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,黄土塘村委,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
514,黄塘桥,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
515,黄家坝,3.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516,黄泥头,2.0,5.0,9.0,6.0,5.0,5.0,2.0,0.0,5.0,...,0.0,3.0,4.0,5.0,1.0,2.0,0.0,1.0,0.0,0.0
